In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm

In [2]:
segments_dir = "../../../../../data/SPL"
output_path = "/home/imonge/Documents/annotations/ship"
df_ship = pd.read_pickle(os.path.join(segments_dir, 'psd_aplose_ship_only.pkl'))
df_no_ship = pd.read_pickle(os.path.join(segments_dir, 'psd_aplose_no_ship.pkl'))

In [3]:
# Mean amplitude across all frequencies - Scatter plot
%matplotlib qt
mean_amp_ship = df_ship.mean(axis=1)
mean_amp_no_ship = df_no_ship.mean(axis=1)

# Convert to dB
mean_dB_ship = 10 * np.log10(mean_amp_ship)
mean_dB_no_ship = 10 * np.log10(mean_amp_no_ship)

# Plot
# plt.style.use('default')
plt.figure(figsize=(12,6))

plt.scatter(mean_dB_ship.index, mean_dB_ship, label='With ship', color='darkcyan', alpha=0.5)
plt.scatter(mean_dB_no_ship.index, mean_dB_no_ship, label='Without ship', color='darkviolet', alpha=0.7)

plt.xlabel('Date')
plt.ylabel('Mean amplitude (dB re 1 µPa)')
plt.legend()
plt.grid(True, alpha=0.3, linestyle='--', color='gray')
plt.tight_layout()
# plt.show()

plt.savefig(os.path.join(output_path, 'scatterplot_amp.png'))

In [4]:
# Mean amplitude comparison - Box plot
data = [mean_dB_ship, mean_dB_no_ship]
labels = ['With ship', 'Without ship']
colors = ['darkcyan', 'darkviolet']

# Box plot
plt.figure(figsize=(8, 5))
box = plt.boxplot(data, labels=labels, patch_artist=True, showmeans=True, meanline=True)

# Apply colors
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.4)

for median in box['medians']:
    median.set_color('black')

for mean in box['means']:
    mean.set_color('black')

# Add scatter plot
for i, group in enumerate(data, 1): # i = 1 for the fisrt box
    x_jitter = np.random.normal(loc=i - 0.2, scale=0.03, size=len(group))
    plt.scatter(x_jitter, group, alpha=0.5, color=colors[i-1], s=15, edgecolor='k', linewidth=0.3)

plt.ylabel('Mean amplitude (dB re 1 µPa)')
plt.title("Amplitude comparision with and without ship noise")
plt.grid(axis='y', alpha=0.3, linestyle='--', color='gray')
plt.tight_layout()
# plt.show()

plt.savefig(os.path.join(output_path, 'boxplot_amp.png'))

/tmp/ipykernel_91802/1682128494.py:8: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box = plt.boxplot(data, labels=labels, patch_artist=True, showmeans=True, meanline=True)


In [17]:
# Mean amplitude across all frequencies -> savoir si une station est plus bruyante qu'une autre
# Pour que ça fonctionne, il faudrait faire correspondre les indices pour retirer les filenames d'un autre dataframe, puis retirer le numéro de station
%matplotlib qt

# Retrieve station
station_ship = df_ship['station'].unique()
station_no_ship = df_no_ship['station'].unique()

# Create colormaps
color_ship = cm.get_cmap('autumn', len(station_ship))
color_no_ship = cm.get_cmap('Set1', len(station_no_ship))

plt.style.use('default')
plt.figure(figsize=(12,6))

# With ship
for i, station in enumerate(station_ship):
    sub_df = df_ship[df_ship['station'] == station]
    freq_cols = sub_df.select_dtypes(include=np.number).columns
    mean_amp = sub_df[freq_cols].mean(axis=1)
    mean_dB = 10 * np.log10(mean_amp)
    plt.scatter(mean_dB.index, mean_dB, label=f"Ship - {station}", color=color_ship(i), alpha=0.5)

# Without ship
for i, station in enumerate(station_no_ship):
    sub_df = df_no_ship[df_no_ship['station'] == station]
    freq_cols = sub_df.select_dtypes(include=np.number).columns
    mean_amp = sub_df[freq_cols].mean(axis=1)
    mean_dB = 10 * np.log10(mean_amp)
    plt.scatter(mean_dB.index, mean_dB, label=f"No ship - {station}", color=color_no_ship(i), alpha=1, s=50)

plt.xlabel('Date')
plt.ylabel('Mean amplitude (dB re 1 µPa)')
plt.legend()
plt.grid(True, alpha=0.3, linestyle='--', color='gray')
plt.tight_layout()
plt.show()

# plt.savefig(os.path.join(output_path, 'no_ship_amp_stations.png'))

KeyError: 'station'